In [1]:
import pandas as pd
import urllib.request
import json
import dml
import prov.model
import datetime
import uuid

In [236]:
client = dml.pymongo.MongoClient()
repo = client.repo
repo1.authenticate('chuci_yfch_yuwan_zhurh', 'chuci_yfch_yuwan_zhurh')
url = 'http://datamechanics.io/data/uber_newcity_cc.json'
response = urllib.request.urlopen(url).read().decode("utf-8")
r = json.loads(response)
s = json.dumps(r, sort_keys=True, indent=2)
repo1.dropCollection("uber")
repo1.createCollection("uber")
repo1['chuci_yfch_yuwan_zhurh.uber'].insert_many(r)
repo1['chuci_yfch_yuwan_zhurh.uber'].metadata({'complete':True})

In [227]:
url2 = 'http://datamechanics.io/data/health_new_int_cc.json'
response2 = urllib.request.urlopen(url2).read().decode("utf-8")
r2 = json.loads(response2)
s2 = json.dumps(r2, sort_keys=True, indent=2)
repo1.dropCollection("health")
repo1.createCollection("health")
repo1['chuci_yfch_yuwan_zhurh.health'].insert_many(r2)
repo1['chuci_yfch_yuwan_zhurh.health'].metadata({'complete':True})

In [228]:
repo1['chuci_yfch_yuwan_zhurh.health'].find_one()

{'_id': ObjectId('5c7c4fca6cc4b60250144c9b'),
 'StateDesc': 'Washington',
 'CityName': 'Seattle',
 'PopulationCount': 608660}

In [234]:
pipline = [
    { '$group' : { '_id' : "$CityName", 'population': { '$max': "$PopulationCount" } } }
]

agg_result = repo1['chuci_yfch_yuwan_zhurh.health'].aggregate(pipline)

agg_list = list(agg_result)

repo1.dropCollection("population")
repo1.createCollection("population")
repo1['chuci_yfch_yuwan_zhurh.population'].insert_many(agg_list)

In [235]:
list(repo1['chuci_yfch_yuwan_zhurh.population'].find())

[{'_id': 'Los Angeles', 'population': 3792621},
 {'_id': 'Washington', 'population': 601723},
 {'_id': 'Boston', 'population': 617594},
 {'_id': 'Seattle', 'population': 608660}]

In [237]:
repo1['chuci_yfch_yuwan_zhurh.uber'].find_one()

{'_id': ObjectId('5c7c512e6cc4b6025014f088'),
 'CITY': 'Seattle',
 'Origin Movement ID': 259,
 'Origin Display Name': 'Census Tract 75',
 'Destination Movement ID': 1,
 'Destination Display Name': 'Census Tract 220.06',
 'Date Range': '1/1/2018 - 1/31/2018, Every day, Daily Average',
 'Mean Travel Time (Seconds)': 1570,
 'Range - Lower Bound Travel Time (Seconds)': 1309,
 'Range - Upper Bound Travel Time (Seconds)': 1882}

In [238]:
pipline2 = [
    { '$group' : { '_id' : "$CITY", 'mean_time': { '$avg': "$Mean Travel Time (Seconds)" } } }
]

agg_result_uber = repo1['chuci_yfch_yuwan_zhurh.uber'].aggregate(pipline2)

agg_list_uber = list(agg_result_uber)

repo1.dropCollection("uber_mean")
repo1.createCollection("uber_mean")
repo1['chuci_yfch_yuwan_zhurh.uber_mean'].insert_many(agg_list_uber)

In [239]:
list(repo1['chuci_yfch_yuwan_zhurh.uber_mean'].find())

[{'_id': 'Los Angeles', 'mean_time': 1479.9741468459151},
 {'_id': 'Washington', 'mean_time': 1306.3177737881508},
 {'_id': 'Boston', 'mean_time': 1536.976127320955},
 {'_id': 'Seattle', 'mean_time': 1202.306338028169}]

In [271]:
l1 = list(repo1['chuci_yfch_yuwan_zhurh.population'].find())

l2 = list(repo1['chuci_yfch_yuwan_zhurh.uber_mean'].find())

l3 = []
for each1 in l1:
    for each2 in l2:
        if each1['_id'] == each2['_id']:
            l3.append({'_id':each1['_id'],
                      'population':each1['population'],
                       'mean_time':each2['mean_time']
                      })

In [273]:
repo1.dropCollection("city_pop_time")
repo1.createCollection("city_pop_time")
repo1['chuci_yfch_yuwan_zhurh.city_pop_time'].insert_many(l3)